<a href="https://colab.research.google.com/github/nicikess/hsg-nlp-course/blob/main/TEST_huggingface_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from datasets import load_dataset

dataset = load_dataset('csv', data_files='/content/drive/MyDrive/NLP/data/train_sorted.csv')

Using custom data configuration default-5030250fbae37ebd
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-5030250fbae37ebd/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'index', 'nwo', 'path', 'function_name', 'lineno', 'original_function', 'function_tokens', 'docstring_tokens', 'url', 'function_tokens_count'],
        num_rows: 116109
    })
})

Take train out of the dictionary

In [4]:
dataset = dataset['train']

In [5]:
dataset

Dataset({
    features: ['Unnamed: 0', 'index', 'nwo', 'path', 'function_name', 'lineno', 'original_function', 'function_tokens', 'docstring_tokens', 'url', 'function_tokens_count'],
    num_rows: 116109
})

Take out columns that are not used

In [6]:
columns = dataset.column_names
columns_to_keep = ["function_name", "docstring_tokens", "url", "original_function"]
columns_to_remove = set(columns_to_keep).symmetric_difference(columns)
dataset = dataset.remove_columns(columns_to_remove)
dataset

Dataset({
    features: ['function_name', 'original_function', 'docstring_tokens', 'url'],
    num_rows: 116109
})

Inspect data

In [7]:
dataset.set_format("pandas")
df = dataset[:]

In [8]:
df["original_function"][0]

'def channelModes():\n    """\n\t\tReturns the channel modes provided by the module.  The modes are returned as\n\t\ta list of tuples:\n\t\t[ (letter, type, object, rank, symbol) ]\n\t\tThe letter is the letter of the mode.\n\t\tThe type is a mode type, one of ModeType.List, ModeType.ParamOnUnset, ModeType.Param,\n\t\t\tModeType.NoParam, and ModeType.Status.  (ModeType is in txircd.utils.)\n\t\tThe object is an instance of the class that implements the mode.\n\t\tThe rank is a numeric value only relevant for ModeType.Status modes.  Higher numbers\n\t\t\tindicate a higher channel rank.  The default chanop mode +o has a rank of 100, and\n\t\t\tthe default voice mode +v has a rank of 10.\n\t\tThe symbol is a single character, also only relevant for ModeType.Status modes.\n\t\t"""\n'

In [9]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['function_name', 'original_function', 'docstring_tokens', 'url'],
    num_rows: 116109
})

In [10]:
dataset.features

{'docstring_tokens': Value(dtype='string', id=None),
 'function_name': Value(dtype='string', id=None),
 'original_function': Value(dtype='string', id=None),
 'url': Value(dtype='string', id=None)}

In [11]:
def concatenate_text(examples):
  return {
    "text": str(examples["function_name"])
    + " \n "
    + str(examples["docstring_tokens"])
    + " \n "
    + str(examples["original_function"])
  }

function_dataset = dataset.map(concatenate_text)

  0%|          | 0/116109 [00:00<?, ?ex/s]

In [12]:
function_dataset["text"][0]

'channelModes \n returns the channel modes provided by the module the modes are returned as a list of tuples \n def channelModes():\n    """\n\t\tReturns the channel modes provided by the module.  The modes are returned as\n\t\ta list of tuples:\n\t\t[ (letter, type, object, rank, symbol) ]\n\t\tThe letter is the letter of the mode.\n\t\tThe type is a mode type, one of ModeType.List, ModeType.ParamOnUnset, ModeType.Param,\n\t\t\tModeType.NoParam, and ModeType.Status.  (ModeType is in txircd.utils.)\n\t\tThe object is an instance of the class that implements the mode.\n\t\tThe rank is a numeric value only relevant for ModeType.Status modes.  Higher numbers\n\t\t\tindicate a higher channel rank.  The default chanop mode +o has a rank of 100, and\n\t\t\tthe default voice mode +v has a rank of 10.\n\t\tThe symbol is a single character, also only relevant for ModeType.Status modes.\n\t\t"""\n'

In [13]:
from transformers import AutoTokenizer, AutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [14]:
import torch

device = torch.device("cuda")
model.to(device)

In [15]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [16]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [17]:
embedding = get_embeddings(function_dataset["text"][0])
embedding.shape

torch.Size([1, 768])

In [ ]:
embeddings_dataset = function_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).detach().cpu().numpy()[0]}
)

  0%|          | 0/116109 [00:00<?, ?ex/s]